### Imports

In [38]:
import torch
from torch import autograd

### Tensors and Variables
Declare a torch tensor of the size mentioned as argument. This is auto-initialized with zeros.

In [13]:
x = torch.Tensor(2)
y = torch.Tensor(4, 2)
print(x)
print(y)

tensor([0.0000, 0.0000])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]])


In [16]:
print(len(y))

4


#### Declaring a tensor with random values

In [21]:
randx = torch.rand(2, 3)
randx

tensor([[0.4711, 0.5733, 0.3298],
        [0.8608, 0.7944, 0.7181]])

#### Declaring a tensor with ones

In [32]:
onex = torch.ones(2,3)
onex

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [24]:
onex*2

tensor([[2., 2., 2.],
        [2., 2., 2.]])

In [33]:
onex[:, 1] = onex[:, 1]*2
onex

tensor([[1., 2., 1.],
        [1., 2., 1.]])

### Autograd

Creating a variable from a tensor

In [51]:
x = autograd.Variable(torch.ones(2, 2) * 2, requires_grad=True)
x

tensor([[2., 2.],
        [2., 2.]], requires_grad=True)

In [52]:
z = 2 * (x * x) + 5 * x
z

tensor([[18., 18.],
        [18., 18.]], grad_fn=<ThAddBackward>)

To compute the gradient, we need to compute it against a vector x to get a d/dx

In [53]:
z.backward(torch.ones(2, 2))
print(x.grad)

tensor([[13., 13.],
        [13., 13.]])


This is the result we get after taking the derivative of 2x^2 + 5x i.e. 4x + 5

### Neural Network Class

In [54]:
import torch.nn as nn
import torch.nn.functional as F

In [94]:
# Inheriting from the nn.Module class
class Net(nn.Module):
    def __init__(self):
        # "super" creates an instance of the base nn.Module class
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)

A fully connected neural network layer is represented by the nn.Linear object, with the first argument in the definition being the number of nodes in layer l and the next argument being the number of nodes in layer l+1. As you can observe, the first layer takes the 28 x 28 input pixels and connects to the first 200 node hidden layer. Then we have another 200 to 200 hidden layer, and finally a connection between the last hidden layer and the output layer.

### Visualizing the network

In [96]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


### Optimizer and Loss

In [97]:
from torch import optim

learning_rate = 0.01
momentum = 0.9

# create a stochastic gradient descent optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

# create a Negative Log Likelihood function
criterion = nn.NLLLoss()

### Loading the data - MNIST Dataset <br /> 
we'll also need DataLoaders for the dataset. This is where TorchVision comes into play. It let's use load the MNIST dataset in a handy way. We'll use a batch_size of 64 for training and size 1000 for testing on this dataset. The values 0.1307 and 0.3081 used for the Normalize() transformation below are the global mean and standard deviation of the MNIST dataset, we'll take them as a given here.

In [98]:
import torchvision

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
log_interval = 10
random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

# Training data set
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist_train/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

# Testing data set
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./mnist_train/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

### Running the Training Loop

In [99]:
log_interval=10
epochs=10
log_interval=10

# run the main training loop
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = autograd.Variable(data), autograd.Variable(target)
        # resize data from (batch_size, 1, 28, 28) to (batch_size, 28*28)
        data = data.view(-1, 28*28)
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data[0]))

# run a test loop
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = autograd.Variable(data, volatile=True), autograd.Variable(target)
    data = data.view(-1, 28 * 28)
    net_out = net(data)
    # sum up batch loss
    test_loss += criterion(net_out, target).data[0]
    pred = net_out.data.max(1)[1]  # get the index of the max log-probability
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

C:\Users\shash\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  
C:\Users\shash\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.304721
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.207470
Train Epoch: 0 [1280/60000 (2%)]	Loss: 1.920889
Train Epoch: 0 [1920/60000 (3%)]	Loss: 1.488192
Train Epoch: 0 [2560/60000 (4%)]	Loss: 0.871500
Train Epoch: 0 [3200/60000 (5%)]	Loss: 0.523356
Train Epoch: 0 [3840/60000 (6%)]	Loss: 0.690306
Train Epoch: 0 [4480/60000 (7%)]	Loss: 0.662203
Train Epoch: 0 [5120/60000 (9%)]	Loss: 0.246633
Train Epoch: 0 [5760/60000 (10%)]	Loss: 0.350085
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.385245
Train Epoch: 0 [7040/60000 (12%)]	Loss: 0.443924
Train Epoch: 0 [7680/60000 (13%)]	Loss: 0.551469
Train Epoch: 0 [8320/60000 (14%)]	Loss: 0.440651
Train Epoch: 0 [8960/60000 (15%)]	Loss: 0.363155
Train Epoch: 0 [9600/60000 (16%)]	Loss: 0.336598
Train Epoch: 0 [10240/60000 (17%)]	Loss: 0.302462
Train Epoch: 0 [10880/60000 (18%)]	Loss: 0.268500
Train Epoch: 0 [11520/60000 (19%)]	Loss: 0.260498
Train Epoch: 0 [12160/60000 (20%)]	Loss: 0.182148
Train Epoch: 0 [12800/60000 (

Train Epoch: 1 [45440/60000 (76%)]	Loss: 0.177757
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.107933
Train Epoch: 1 [46720/60000 (78%)]	Loss: 0.192665
Train Epoch: 1 [47360/60000 (79%)]	Loss: 0.080491
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.101255
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.262099
Train Epoch: 1 [49280/60000 (82%)]	Loss: 0.094377
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.064731
Train Epoch: 1 [50560/60000 (84%)]	Loss: 0.040056
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.077162
Train Epoch: 1 [51840/60000 (86%)]	Loss: 0.251171
Train Epoch: 1 [52480/60000 (87%)]	Loss: 0.049988
Train Epoch: 1 [53120/60000 (88%)]	Loss: 0.038810
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.175979
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.180709
Train Epoch: 1 [55040/60000 (92%)]	Loss: 0.021787
Train Epoch: 1 [55680/60000 (93%)]	Loss: 0.185845
Train Epoch: 1 [56320/60000 (94%)]	Loss: 0.022722
Train Epoch: 1 [56960/60000 (95%)]	Loss: 0.024738
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.093246


Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.017314
Train Epoch: 3 [31360/60000 (52%)]	Loss: 0.083252
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.051881
Train Epoch: 3 [32640/60000 (54%)]	Loss: 0.030116
Train Epoch: 3 [33280/60000 (55%)]	Loss: 0.008871
Train Epoch: 3 [33920/60000 (57%)]	Loss: 0.019982
Train Epoch: 3 [34560/60000 (58%)]	Loss: 0.026729
Train Epoch: 3 [35200/60000 (59%)]	Loss: 0.021504
Train Epoch: 3 [35840/60000 (60%)]	Loss: 0.056253
Train Epoch: 3 [36480/60000 (61%)]	Loss: 0.066879
Train Epoch: 3 [37120/60000 (62%)]	Loss: 0.022523
Train Epoch: 3 [37760/60000 (63%)]	Loss: 0.021910
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.065453
Train Epoch: 3 [39040/60000 (65%)]	Loss: 0.021446
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.021193
Train Epoch: 3 [40320/60000 (67%)]	Loss: 0.094900
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.012898
Train Epoch: 3 [41600/60000 (69%)]	Loss: 0.007531
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.052703
Train Epoch: 3 [42880/60000 (71%)]	Loss: 0.040132


Train Epoch: 5 [16000/60000 (27%)]	Loss: 0.024038
Train Epoch: 5 [16640/60000 (28%)]	Loss: 0.005457
Train Epoch: 5 [17280/60000 (29%)]	Loss: 0.014137
Train Epoch: 5 [17920/60000 (30%)]	Loss: 0.005693
Train Epoch: 5 [18560/60000 (31%)]	Loss: 0.043360
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.011097
Train Epoch: 5 [19840/60000 (33%)]	Loss: 0.099081
Train Epoch: 5 [20480/60000 (34%)]	Loss: 0.014962
Train Epoch: 5 [21120/60000 (35%)]	Loss: 0.072005
Train Epoch: 5 [21760/60000 (36%)]	Loss: 0.003660
Train Epoch: 5 [22400/60000 (37%)]	Loss: 0.053274
Train Epoch: 5 [23040/60000 (38%)]	Loss: 0.033874
Train Epoch: 5 [23680/60000 (39%)]	Loss: 0.036637
Train Epoch: 5 [24320/60000 (41%)]	Loss: 0.021687
Train Epoch: 5 [24960/60000 (42%)]	Loss: 0.008242
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.020488
Train Epoch: 5 [26240/60000 (44%)]	Loss: 0.013113
Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.037619
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.033598
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.056859


Train Epoch: 7 [640/60000 (1%)]	Loss: 0.005022
Train Epoch: 7 [1280/60000 (2%)]	Loss: 0.016863
Train Epoch: 7 [1920/60000 (3%)]	Loss: 0.007985
Train Epoch: 7 [2560/60000 (4%)]	Loss: 0.005849
Train Epoch: 7 [3200/60000 (5%)]	Loss: 0.001297
Train Epoch: 7 [3840/60000 (6%)]	Loss: 0.024014
Train Epoch: 7 [4480/60000 (7%)]	Loss: 0.022334
Train Epoch: 7 [5120/60000 (9%)]	Loss: 0.013402
Train Epoch: 7 [5760/60000 (10%)]	Loss: 0.005937
Train Epoch: 7 [6400/60000 (11%)]	Loss: 0.016536
Train Epoch: 7 [7040/60000 (12%)]	Loss: 0.010812
Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.003613
Train Epoch: 7 [8320/60000 (14%)]	Loss: 0.008266
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.000337
Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.000813
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.015836
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.009812
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.021455
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.004840
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.000552
Train Epoch: 7 [13440/60

Train Epoch: 8 [45440/60000 (76%)]	Loss: 0.006192
Train Epoch: 8 [46080/60000 (77%)]	Loss: 0.005321
Train Epoch: 8 [46720/60000 (78%)]	Loss: 0.017118
Train Epoch: 8 [47360/60000 (79%)]	Loss: 0.004504
Train Epoch: 8 [48000/60000 (80%)]	Loss: 0.005415
Train Epoch: 8 [48640/60000 (81%)]	Loss: 0.006643
Train Epoch: 8 [49280/60000 (82%)]	Loss: 0.006833
Train Epoch: 8 [49920/60000 (83%)]	Loss: 0.031733
Train Epoch: 8 [50560/60000 (84%)]	Loss: 0.004452
Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.007860
Train Epoch: 8 [51840/60000 (86%)]	Loss: 0.017443
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.006232
Train Epoch: 8 [53120/60000 (88%)]	Loss: 0.008127
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.015937
Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.041298
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.011152
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.001117
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.005239
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.002150
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.001830


C:\Users\shash\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Users\shash\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0001, Accuracy: 9817/10000 (98%)

